### using static (non-trainable) glove vectors with convolutional layers to classify newsgroup_20 documents

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html


In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D
from keras.models import Model

Using TensorFlow backend.


In [2]:
np.random.seed(seed=42)

In [3]:
texts,labels_index,labels = files.read_20_newgroup_files()        

In [4]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

In [5]:
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
X_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [6]:
data.shape,labels.shape

((19997, 1000), (19997, 20))

In [7]:
embeddings_index = files.read_glove_100()

In [8]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [9]:
embedding_layer = Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False)

In [10]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

In [11]:
model = Model(sequence_input, preds)
model.compile(loss ='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['acc'])

In [12]:
%%time
model.fit(X_train,y_train, validation_data=(X_val, y_val),
         epochs=10, batch_size=32)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 19s - loss: 2.2225 - acc: 0.2353 - val_loss: 1.5649 - val_acc: 0.4274
Epoch 2/10
15998/15998 [==============================] - 18s - loss: 1.3129 - acc: 0.5388 - val_loss: 1.2015 - val_acc: 0.5916
Epoch 3/10
15998/15998 [==============================] - 18s - loss: 0.9689 - acc: 0.6713 - val_loss: 1.0236 - val_acc: 0.6672
Epoch 4/10
15998/15998 [==============================] - 18s - loss: 0.7621 - acc: 0.7455 - val_loss: 1.0007 - val_acc: 0.6779
Epoch 5/10
15998/15998 [==============================] - 18s - loss: 0.6027 - acc: 0.8004 - val_loss: 0.8884 - val_acc: 0.7197
Epoch 6/10
15998/15998 [==============================] - 18s - loss: 0.4892 - acc: 0.8392 - val_loss: 0.9647 - val_acc: 0.7212
Epoch 7/10
15998/15998 [==============================] - 18s - loss: 0.4023 - acc: 0.8627 - val_loss: 0.9613 - val_acc: 0.7332
Epoch 8/10
15998/15998 [==============================]